In [7]:
import polars as pl
import pandas as pd

In [8]:
from IPython import get_ipython
get_ipython().run_line_magic("run", "01_pagos_recup_functions.ipynb")

In [9]:
import time
start_time = time.time()

In [10]:
cierre = "202506"
cierre_l = "Jun25"

# Carpetas
path = "E:/Users/jhernandezr/DAR/garantias/reporte/fotos/"
wd_data_raw = path + "data/raw/"
wd_data_external = path + "data/external/"
wd_data_processed_dwh = "E:/Users/jhernandezr/DAR/garantias/data_pipeline_garantias/data/processed/DWH/"


# Archivos
# Inputs
fl_catalogos = wd_data_external + f'Catálogos_{cierre_l}.accdb'
fl_saldos = wd_data_raw + f"Saldos_{cierre}.mdb"
fl_saldos_bmxt = wd_data_raw + f"Saldos_BMXT_{cierre}.mdb"
fl_saldos_fianzas = wd_data_raw + f"Saldos_80686_{cierre}.mdb"

# Outputs
fl_saldos_totales_vf = wd_data_processed_dwh + f'parquet/DWH_Saldos_Totales_{cierre}_VF.parquet'
fl_agrup_saldo_total = wd_data_processed_dwh + f'parquet/BD_DWH_Agrup_SaldoTotal_{cierre}.parquet'
fl_agrup_taxo_saldo_total = wd_data_processed_dwh + f'parquet/BD_DWH_AgrupTaxo_SaldoTotal_{cierre}.parquet'

In [11]:
schema_saldos = {
    'DESC_INDICADOR': pl.String,
    'FECHA_CONSULTA': pl.Date,
    'HISTORICO': pl.String,
    'INDICADOR_ID': pl.Int32,
    'INTERMEDIARIO_ID': pl.String,
    'PROGRAMA_DESCRIPCION': pl.String,
    'PROGRAMA_ID': pl.Int32,
    'PROGRAMA_ORIGINAL': pl.Int32,
    'RAZON_SOCIAL': pl.String,
    'TIPO_CREDITO_ID': pl.Int16,
    'Número de registros': pl.Int16,
    'SALDO_CONTINGENTE_MN (SUMA)': pl.Float64
}

In [12]:
def import_saldos_totales():
    df_saldos = read_access(fl_saldos, "DATOS")
    df_saldos_bmxt = read_access(fl_saldos_bmxt, "DATOS")
    df_saldos_fianzas = read_access(fl_saldos_fianzas, "DATOS")

    df_saldos_totales = pl.concat([
        df_saldos, 
        df_saldos_bmxt, 
        df_saldos_fianzas
        ], rechunk=True)
    
    df_saldos_totales = (
        df_saldos_totales.select(
            pl.col("DESC_INDICADOR").alias("Producto"),
            pl.col("FECHA_CONSULTA"),
            pl.col("INDICADOR_ID").alias("Producto ID"),
            pl.col("INTERMEDIARIO_ID"),
            pl.col("PROGRAMA_DESCRIPCION").alias("Programa Descripción (ID)"),
            pl.col("PROGRAMA_ID"),
            pl.col("PROGRAMA_ORIGINAL"),
            pl.col("RAZON_SOCIAL").alias("Razón Social (Intermediario)"),
            pl.col("TIPO_CREDITO_ID"),
            pl.col("SALDO_CONTINGENTE_MN (SUMA)").alias("SUM Saldo_Contingente_Mn")
        )
    )
    df_saldos_totales = genera_tpro_clave(df_saldos_totales)
    return df_saldos_totales

df_saldos_totales = import_saldos_totales()

In [13]:
def genera_saldos_totales_vf(df):
    tipo_credito = read_access(fl_catalogos, "TIPO_CREDITO")
    tipo_credito = tipo_credito.cast({'Tipo_Credito_ID': pl.Int32})

    df_saldos_totales_vf = (
        df.join(tipo_credito.select(['Tipo_Credito_ID', 'NR_R']),
                            left_on='TIPO_CREDITO_ID', right_on='Tipo_Credito_ID',
                            how='left')
    )
    return df_saldos_totales_vf

saldos_totales_vf = genera_saldos_totales_vf(df_saldos_totales)

In [14]:
# Agrup saldo total
agrup_saldo_total = (
    saldos_totales_vf
    .group_by([
        "Producto", 
        "Razón Social (Intermediario)", 
        "INTERMEDIARIO_ID", 
        "PROGRAMA_ORIGINAL", 
        "PROGRAMA_ID"
        ]).agg(
            pl.col("SUM Saldo_Contingente_Mn").sum().alias("SALDO_MN")
        )
)

# Agrup saldo total resto
(
    saldos_totales_vf
    .filter(
        (pl.col("Producto") == "GARANTIAS BANCOMEXT") |
        (pl.col("Producto") == "GARANTIAS SHF/LI FINANCIERO") |
        (pl.col("Producto") == "GARANTIAS BANSEFI") |
        (pl.col("Razón Social (Intermediario)") == "FISO PARA EL AHORRO DE ENERGIA ELECTRICA (FASE II)")
    ).group_by([
        "Producto", 
        "Razón Social (Intermediario)", 
        "INTERMEDIARIO_ID", 
        "PROGRAMA_ORIGINAL", 
        "PROGRAMA_ID"
        ]).agg(
            pl.col("SUM Saldo_Contingente_Mn").sum().alias("SALDO_MN")
        )
)

Producto,Razón Social (Intermediario),INTERMEDIARIO_ID,PROGRAMA_ORIGINAL,PROGRAMA_ID,SALDO_MN
str,str,str,i64,i64,f64
"""GARANTIAS BANCOMEXT""","""BANCOMEXT/BAJIO""","""64991057""",3999,39451,5.5599e6
"""GARANTIAS BANCOMEXT""","""BANCOMEXT / BANSI""","""481346210""",3999,39817,3.33312e6
"""GARANTIAS BANCOMEXT""","""BANCOMEXT/BAJIO""","""64991057""",3999,39467,401028.156
"""GARANTIAS BANCOMEXT""","""BANCOMEXTBANAMEX""","""468297752""",3999,390161,1.1855e8
"""GARANTIAS BANCOMEXT""","""BANCOMEXT/BAJIO""","""64991057""",3999,390194,2.8833e8
…,…,…,…,…,…
"""GARANTIAS BANCOMEXT""","""BANCOMEXT/BANCO REGIONAL DE MO…","""479696360""",3999,39914,1.0379e6
"""GARANTIAS BANCOMEXT""","""BANCOMEXTBANAMEX""","""468297752""",3999,390102,5.6703e7
"""GARANTIAS BANCOMEXT""","""BANCOMEXT/BAJIO""","""64991057""",3999,390058,1.2167e7


In [15]:
# Agrup saldo total automatica
(
    saldos_totales_vf
    .filter(
        (pl.col("Producto") != "GARANTIAS BANCOMEXT") &
        (pl.col("Producto") != "GARANTIAS SHF/LI FINANCIERO") &
        (pl.col("Producto") != "GARANTIAS BANSEFI") &
        (pl.col("Razón Social (Intermediario)") != "FISO PARA EL AHORRO DE ENERGIA ELECTRICA (FASE II)")
    ).group_by([
        "Producto", 
        "Razón Social (Intermediario)", 
        "INTERMEDIARIO_ID", 
        "PROGRAMA_ORIGINAL", 
        "PROGRAMA_ID"
        ]).agg(
            pl.col("SUM Saldo_Contingente_Mn").sum().alias("SALDO_MN")
        )
)

# AgrupTaxo saldo total
agrup_taxo_saldo_total = (
    saldos_totales_vf
    .group_by([
        "Producto"
        ]).agg(
            pl.col("SUM Saldo_Contingente_Mn").sum().alias("SALDO_MN")
        )
)

# AgrupTaxo saldo total resto
(
    saldos_totales_vf
        .filter(
            (pl.col("Producto") == "GARANTIAS BANCOMEXT") |
            (pl.col("Producto") == "GARANTIAS SHF-LI FINANCIERO") |
            (pl.col("Producto") == "GARANTIAS BANSEFI") |
            (pl.col("Razón Social (Intermediario)") == "FISO PARA EL AHORRO DE ENERGIA ELECTRICA (FASE II)")
        ).group_by([
            "Producto"
            ]).agg(
            pl.col("SUM Saldo_Contingente_Mn").sum().alias("SALDO_MN")
        )
)

(
    saldos_totales_vf
    .filter(
        (pl.col("Producto") != "GARANTIAS BANCOMEXT") &
        (pl.col("Producto") != "GARANTIAS SHF/LI FINANCIERO") &
        (pl.col("Producto") != "GARANTIAS BANSEFI") &
        (pl.col("Razón Social (Intermediario)") != "FISO PARA EL AHORRO DE ENERGIA ELECTRICA (FASE II)")
        ).group_by([
            "Producto"
            ]).agg(
            pl.col("SUM Saldo_Contingente_Mn").sum().alias("SALDO_MN")
        )
)

Producto,SALDO_MN
str,f64
"""GARANTIA SUBASTA""",402382.634663
"""GARANTIAS FEFA""",3.4366e8
"""GARANTIA EMPRESARIAL""",9.4991e10
"""EMPRESA MEDIANA""",3.0404e8
"""GARANTIA FIANZA""",6.5166e9
"""GARANTIA SECTORIAL""",1.7708e10
"""GARANTIA EMERGENTES""",4.7175e8
"""GARANTIA ESPECIAL""",6.4603e7


In [16]:
# Exporta CSV
# saldos_totales_vf.write_csv(fl_saldos_totales_vf, datetime_format="%Y-%m-%d", float_precision=6)
# agrup_saldo_total.write_csv(fl_agrup_saldo_total, datetime_format="%Y-%m-%d", float_precision=6)
# agrup_taxo_saldo_total.write_csv(fl_agrup_taxo_saldo_total, datetime_format="%Y-%m-%d", float_precision=6)


In [17]:
# Exporta Parquet
saldos_totales_vf.write_parquet(fl_saldos_totales_vf)
agrup_saldo_total.write_parquet(fl_agrup_saldo_total)
agrup_taxo_saldo_total.write_parquet(fl_agrup_taxo_saldo_total)

In [18]:
end_time = time.time()
end_time - start_time

1.2410268783569336

### macro_pe_p10